In [1]:
import os
import cv2
import mediapipe as mp

In [2]:
def check_frames_per_second(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Không thể mở video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")

    videoLengthInMinutes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

    print(f"Video length in seconds: {videoLengthInMinutes}")

video_path = "plank.mp4"
check_frames_per_second(video_path)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.0
Video length in seconds: 54


Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder

In [3]:
# Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder
def extract_images_from_video(video_path, output_folder, timeStep=0.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")
    
    t = 0
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if t % int(fps * timeStep) == 0:
                count += 1
                cv2.imwrite(output_folder + f"/frame_{count}.jpg", frame)
            t += 1
        else:
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
extract_images_from_video('plank.mp4', 'data/images')

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.0


In [4]:
# Trích xuất các keypoint từ ảnh
def extract_keypoints(image_path):
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)
    keypoints = []
    for data_point in results.pose_landmarks.landmark:
        keypoints.append({
            'x': data_point.x,
            'y': data_point.y,
            'z': data_point.z,
            'visibility': data_point.visibility,
        })
    return keypoints

Hiển thị 1 ảnh kèm keypoint

In [5]:
# Tạo cửa sổ để hiển thị ảnh và keypoint
def display_image(image_path, keypoints):
    image = cv2.imread(image_path)
    for point in keypoints:
        x = int(point['x'] * image.shape[1])
        y = int(point['y'] * image.shape[0])
        cv2.circle(image, (x, y), 3, (0, 0, 255), -1)
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
display_image('data/images/frame_1.jpg', extract_keypoints('data/images/frame_1.jpg'))

Từ tất cả các ảnh trong 1 folder, trích xuất và chuyển vào 1 folder mới

In [7]:
def add_keypoints_to_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        keypoints = extract_keypoints(image_path)

        # Vẽ keypoint lên ảnh và lưu vào folder mới
        image = cv2.imread(image_path)
        for point in keypoints:
            x = int(point['x'] * image.shape[1])
            y = int(point['y'] * image.shape[0])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)

        output_path = os.path.join(output_folder, image_name)  # Corrected line
        cv2.imwrite(output_path, image)  # Corrected line

In [14]:
add_keypoints_to_images('data/images', 'data/images_with_keypoints')

: 

Tool để gán nhãn nhanh

In [8]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QVBoxLayout, QWidget, QPushButton
from PyQt5.QtGui import QPixmap

import os
import shutil

def copy_images_to_folder(input_file, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # nếu trùng tên file thì sẽ bị ghi đè
    shutil.copy(input_file, output_folder)

def window():
    app = QApplication(sys.argv)
    window = QMainWindow()
    window.setGeometry(200, 200, 1000, 800)
    window.setWindowTitle("Plank")
    window.setStyleSheet("background-color: black;")

    images = os.listdir('data/images_with_keypoints')
    imageLen = images.__len__()
    current_image_index = 0
    
    # Tạo label hiển thị tên ảnh
    namePic = QLabel(window)
    namePic.setText(images[current_image_index])
    namePic.setStyleSheet("color: white;")
    namePic.move(50, 700)
    namePic.setGeometry(50, 700, 800, 50)

    # Vẽ ảnh lên màn hình cửa sổ với ảnh từ file
    pixmap = QPixmap('data/images_with_keypoints/' + images[current_image_index])
    label = QLabel(window)
    label.setPixmap(pixmap)
    label.setGeometry(50, 50, 800, 600)
    label.setScaledContents(True)

    def updateUI():
        nonlocal current_image_index
        pixmap.load('data/images_with_keypoints/' + images[current_image_index])
        label.setPixmap(pixmap)
        label.repaint()

        namePic.setText(str(current_image_index + 1) + "/" + str(imageLen) + " - " + images[current_image_index])

    updateUI()

    # Tạo button để hiển thị ảnh
    continueButton = QPushButton(window)
    continueButton.setText("Continue")
    continueButton.setStyleSheet("background-color: white;")
    continueButton.move(900, 700)

    # Thêm sự kiện khi click vào button
    def continueButtonClicked():
        nonlocal current_image_index
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0
        
        updateUI()
        
    continueButton.clicked.connect(continueButtonClicked)

    backButton = QPushButton(window)
    backButton.setText("Back")
    backButton.setStyleSheet("background-color: white;")
    backButton.move(600, 700)

    # Thêm sự kiện khi click vào button
    def backButtonClicked():
        nonlocal current_image_index
        current_image_index -= 1
        if current_image_index < 0:
            current_image_index = len(images) - 1
        
        updateUI()

    backButton.clicked.connect(backButtonClicked)

    labelStatus = QLabel(window)
    labelStatus.setText("Copied image to folder: c, w")
    labelStatus.setStyleSheet("color: white;")
    labelStatus.setGeometry(50, 750, 800, 50)

    # Thêm sự kiện khi nhấn nút "c" để copy ảnh hiện tại vào folder khác
    key_c = 67
    catalog_c = 'Correct'
    key_w = 87
    catalog_w = 'Wrong'
    key_u = 85
    catalog_u = 'Up'
    def keyPressEvent(event):
        nonlocal current_image_index
        
        switcher = {
            key_c: catalog_c,
            key_w: catalog_w,
            key_u: catalog_u
        }

        catalog = switcher.get(event.key(), "Another folder")
        copy_images_to_folder('data/images_with_keypoints/' + images[current_image_index], 'data/selected_images' + '/' + catalog)
        labelStatus.setText("Copied " + images[current_image_index] +  " to folder: " + catalog)

        # to next image
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0

        updateUI()

    window.keyPressEvent = keyPressEvent

    window.show()
    try:
        sys.exit(app.exec_())
    except:
        print("Exiting")

window()

Exiting
